In [1]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

In [2]:
mp_holistic = mp.solutions.holistic 
mp_drawing = mp.solutions.drawing_utils 

In [3]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) # COLOR CONVERSION BGR 2 RGB
    image.flags.writeable = False                  # Image is no longer writeable
    results = model.process(image)                 # Make prediction
    image.flags.writeable = True                   # Image is now writeable 
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR) # COLOR COVERSION RGB 2 BGR
    return image, results

In [4]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS) # Draw right hand connections

In [5]:
def draw_styled_landmarks(image, results):
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(121,22,76), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(121,44,250), thickness=2, circle_radius=2)
                             ) 
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2)
                             ) 

In [6]:
def extract_keypoints(results):
    lh = np.array([[res.x, res.y, res.z] for res in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3)
    rh = np.array([[res.x, res.y, res.z] for res in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3)
    return np.concatenate([lh, rh])

In [7]:
# Path for exported data, numpy arrays
DATA_PATH = os.path.join('MP_Data') 

# Actions that we try to detect
actions = np.array(['0','1','2','3','4','5','6','7','8','9','A','B','C','D','W','X','Y','Z','Hello','Thanks','I Love You','Thumbs Up','Thumbs down','Me','You','Namaste'])

# Thirty videos worth of data
no_sequences = 30

# Videos are going to be 30 frames in length
sequence_length = 30

# Folder start
start_folder = 30

In [8]:
for action in actions: 
    for sequence in range(no_sequences):
        try: 
            os.makedirs(os.path.join(DATA_PATH, action, str(sequence)))
        except:
            pass

In [9]:
cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    # NEW LOOP
    # Loop through actions
    for action in actions:
        # Loop through sequences aka videos
        for sequence in range(no_sequences):
            # Loop through video length aka sequence length
            for frame_num in range(sequence_length):

                # Read feed
                ret, frame = cap.read()

                # Make detections
                image, results = mediapipe_detection(frame, holistic)

                # Draw landmarks
                draw_styled_landmarks(image, results)
                
                # NEW Apply wait logic
                if frame_num == 0: 
                    cv2.putText(image, 'STARTING COLLECTION', (120,200), 
                               cv2.FONT_HERSHEY_SIMPLEX, 1, (0,255, 0), 4, cv2.LINE_AA)
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                    cv2.waitKey(500)
                else: 
                    cv2.putText(image, 'Collecting frames for {} Video Number {}'.format(action, sequence), (15,12), 
                               cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1, cv2.LINE_AA)
                    # Show to screen
                    cv2.imshow('OpenCV Feed', image)
                
                # NEW Export keypoints
                keypoints = extract_keypoints(results)
                npy_path = os.path.join(DATA_PATH, action, str(sequence), str(frame_num))
                np.save(npy_path, keypoints)

                # Break gracefully
                if cv2.waitKey(10) & 0xFF == ord('q'):
                    break
                    
    cap.release()
    cv2.destroyAllWindows()

In [9]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

In [10]:
label_map = {label:num for num, label in enumerate(actions)}

In [11]:
sequences, labels = [], []
for action in actions:
    for sequence in np.array(os.listdir(os.path.join(DATA_PATH, action))).astype(int):
        window = []
        for frame_num in range(sequence_length):
            res = np.load(os.path.join(DATA_PATH, action, str(sequence), "{}.npy".format(frame_num)))
            window.append(res)
        sequences.append(window)
        labels.append(label_map[action])

In [12]:
np.array(sequences).shape

(780, 30, 126)

In [13]:
np.array(labels).shape

(780,)

In [14]:

X = np.array(sequences)

In [15]:
X.shape

(780, 30, 126)

In [16]:
y = to_categorical(labels).astype(int)

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05)

In [18]:
y_test.shape

(39, 26)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

In [20]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)

In [21]:
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(30,126)))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(actions.shape[0], activation='softmax'))

In [153]:
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

In [165]:
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

Epoch 1/2000
24/24 [==============================] - 12s 259ms/step - loss: 3.2470 - categorical_accuracy: 0.0337
Epoch 2/2000
24/24 [==============================] - 3s 105ms/step - loss: 3.1204 - categorical_accuracy: 0.0675
Epoch 3/2000
24/24 [==============================] - 3s 129ms/step - loss: 2.9134 - categorical_accuracy: 0.1026
Epoch 4/2000
24/24 [==============================] - 3s 133ms/step - loss: 2.7172 - categorical_accuracy: 0.1093
Epoch 5/2000
24/24 [==============================] - 2s 100ms/step - loss: 2.5093 - categorical_accuracy: 0.1619
Epoch 6/2000
24/24 [==============================] - 3s 135ms/step - loss: 2.4929 - categorical_accuracy: 0.1714
Epoch 7/2000
24/24 [==============================] - 2s 81ms/step - loss: 2.3847 - categorical_accuracy: 0.2051
Epoch 8/2000
24/24 [==============================] - 3s 132ms/step - loss: 2.2964 - categorical_accuracy: 0.2159
Epoch 9/2000
24/24 [==============================] - 3s 118ms/step - loss: 2.2189 - cat

24/24 [==============================] - 2s 83ms/step - loss: 0.2195 - categorical_accuracy: 0.9163
Epoch 144/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.1741 - categorical_accuracy: 0.9352
Epoch 145/2000
24/24 [==============================] - 2s 101ms/step - loss: 0.2069 - categorical_accuracy: 0.9244
Epoch 146/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.1724 - categorical_accuracy: 0.9366
Epoch 147/2000
24/24 [==============================] - 3s 131ms/step - loss: 0.1641 - categorical_accuracy: 0.9366
Epoch 148/2000
24/24 [==============================] - 3s 131ms/step - loss: 0.2471 - categorical_accuracy: 0.9028
Epoch 149/2000
24/24 [==============================] - 3s 132ms/step - loss: 0.3268 - categorical_accuracy: 0.8826
Epoch 150/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.4645 - categorical_accuracy: 0.8529
Epoch 151/2000
24/24 [==============================] - 3s 131ms/step - loss: 0.2067 - c

24/24 [==============================] - 3s 131ms/step - loss: 1.0608 - categorical_accuracy: 0.6883
Epoch 284/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.4090 - categorical_accuracy: 0.8880
Epoch 285/2000
24/24 [==============================] - 3s 131ms/step - loss: 0.1298 - categorical_accuracy: 0.9771
Epoch 286/2000
24/24 [==============================] - 3s 133ms/step - loss: 0.0483 - categorical_accuracy: 0.9919
Epoch 287/2000
24/24 [==============================] - 3s 113ms/step - loss: 0.0667 - categorical_accuracy: 0.9811
Epoch 288/2000
24/24 [==============================] - 2s 96ms/step - loss: 0.0625 - categorical_accuracy: 0.9798
Epoch 289/2000
24/24 [==============================] - 2s 81ms/step - loss: 2.0992 - categorical_accuracy: 0.8084
Epoch 290/2000
24/24 [==============================] - 2s 93ms/step - loss: 2.8574 - categorical_accuracy: 0.2186
Epoch 291/2000
24/24 [==============================] - 3s 138ms/step - loss: 1.7995 - cat

24/24 [==============================] - 3s 131ms/step - loss: 0.1705 - categorical_accuracy: 0.9433
Epoch 424/2000
24/24 [==============================] - 3s 131ms/step - loss: 0.1358 - categorical_accuracy: 0.9609
Epoch 425/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.0982 - categorical_accuracy: 0.9663
Epoch 426/2000
24/24 [==============================] - 3s 133ms/step - loss: 0.0381 - categorical_accuracy: 0.9879
Epoch 427/2000
24/24 [==============================] - 2s 98ms/step - loss: 0.0181 - categorical_accuracy: 0.9973
Epoch 428/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.0096 - categorical_accuracy: 0.9987
Epoch 429/2000
24/24 [==============================] - 2s 89ms/step - loss: 0.0056 - categorical_accuracy: 1.0000
Epoch 430/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.0042 - categorical_accuracy: 1.0000
Epoch 431/2000
24/24 [==============================] - 3s 129ms/step - loss: 0.0037 - ca

24/24 [==============================] - 3s 132ms/step - loss: 2.6751e-05 - categorical_accuracy: 1.0000
Epoch 561/2000
24/24 [==============================] - 3s 130ms/step - loss: 2.6171e-05 - categorical_accuracy: 1.0000
Epoch 562/2000
24/24 [==============================] - 3s 123ms/step - loss: 2.5698e-05 - categorical_accuracy: 1.0000
Epoch 563/2000
24/24 [==============================] - 3s 131ms/step - loss: 2.5278e-05 - categorical_accuracy: 1.0000
Epoch 564/2000
24/24 [==============================] - 3s 131ms/step - loss: 2.4814e-05 - categorical_accuracy: 1.0000
Epoch 565/2000
24/24 [==============================] - 3s 132ms/step - loss: 2.4312e-05 - categorical_accuracy: 1.0000
Epoch 566/2000
24/24 [==============================] - 3s 131ms/step - loss: 2.3856e-05 - categorical_accuracy: 1.0000
Epoch 567/2000
24/24 [==============================] - 3s 131ms/step - loss: 2.3369e-05 - categorical_accuracy: 1.0000
Epoch 568/2000
24/24 [==============================] -

...

Epoch 1452/2000
24/24 [==============================] - 3s 116ms/step - loss: 4.6851e-05 - categorical_accuracy: 1.0000
Epoch 1453/2000
24/24 [==============================] - 2s 90ms/step - loss: 4.6229e-05 - categorical_accuracy: 1.0000
Epoch 1454/2000
24/24 [==============================] - 3s 129ms/step - loss: 4.4735e-05 - categorical_accuracy: 1.0000
Epoch 1455/2000
24/24 [==============================] - 3s 130ms/step - loss: 6.7971e-05 - categorical_accuracy: 1.0000
Epoch 1456/2000
24/24 [==============================] - 3s 131ms/step - loss: 0.3407 - categorical_accuracy: 0.9393
Epoch 1457/2000
24/24 [==============================] - 3s 130ms/step - loss: 1.5267 - categorical_accuracy: 0.6491
Epoch 1458/2000
24/24 [==============================] - 2s 94ms/step - loss: 0.5218 - categorical_accuracy: 0.8057
Epoch 1459/2000
24/24 [==============================] - 2s 87ms/step - loss: 0.2152 - categorical_accuracy: 0.9339
Epoch 1460/2000
24/24 [============================

24/24 [==============================] - 3s 111ms/step - loss: 1.1538e-04 - categorical_accuracy: 1.0000
Epoch 1589/2000
24/24 [==============================] - 2s 91ms/step - loss: 1.1066e-04 - categorical_accuracy: 1.0000
Epoch 1590/2000
24/24 [==============================] - 3s 131ms/step - loss: 1.0825e-04 - categorical_accuracy: 1.0000
Epoch 1591/2000
24/24 [==============================] - 3s 130ms/step - loss: 1.0488e-04 - categorical_accuracy: 1.0000
Epoch 1592/2000
24/24 [==============================] - 3s 132ms/step - loss: 1.0251e-04 - categorical_accuracy: 1.0000
Epoch 1593/2000
24/24 [==============================] - 3s 132ms/step - loss: 1.0010e-04 - categorical_accuracy: 1.0000
Epoch 1594/2000
24/24 [==============================] - 3s 130ms/step - loss: 9.8326e-05 - categorical_accuracy: 1.0000
Epoch 1595/2000
24/24 [==============================] - 2s 85ms/step - loss: 9.6119e-05 - categorical_accuracy: 1.0000
Epoch 1596/2000
24/24 [===========================

24/24 [==============================] - 2s 87ms/step - loss: 0.0267 - categorical_accuracy: 0.9933
Epoch 1724/2000
24/24 [==============================] - 3s 131ms/step - loss: 0.0306 - categorical_accuracy: 0.9879
Epoch 1725/2000
24/24 [==============================] - 3s 120ms/step - loss: 0.0537 - categorical_accuracy: 0.9757
Epoch 1726/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.0487 - categorical_accuracy: 0.9852
Epoch 1727/2000
24/24 [==============================] - 3s 129ms/step - loss: 0.2780 - categorical_accuracy: 0.9109
Epoch 1728/2000
24/24 [==============================] - 3s 131ms/step - loss: 0.2282 - categorical_accuracy: 0.9258
Epoch 1729/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.2753 - categorical_accuracy: 0.9217
Epoch 1730/2000
24/24 [==============================] - 3s 125ms/step - loss: 0.1010 - categorical_accuracy: 0.9690
Epoch 1731/2000
24/24 [==============================] - 3s 119ms/step - loss: 0.

Epoch 1793/2000
24/24 [==============================] - 3s 116ms/step - loss: 0.0015 - categorical_accuracy: 1.0000
Epoch 1794/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.0015 - categorical_accuracy: 1.0000
Epoch 1795/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.0013 - categorical_accuracy: 1.0000
Epoch 1796/2000
24/24 [==============================] - 3s 129ms/step - loss: 0.0013 - categorical_accuracy: 1.0000
Epoch 1797/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.0012 - categorical_accuracy: 1.0000
Epoch 1798/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.0011 - categorical_accuracy: 1.0000
Epoch 1799/2000
24/24 [==============================] - 3s 130ms/step - loss: 0.0011 - categorical_accuracy: 1.0000
Epoch 1800/2000
24/24 [==============================] - 3s 125ms/step - loss: 0.0010 - categorical_accuracy: 1.0000
Epoch 1801/2000
24/24 [==============================] - 3s 130m

24/24 [==============================] - 3s 116ms/step - loss: 8.0968e-04 - categorical_accuracy: 1.0000
Epoch 1930/2000
24/24 [==============================] - 3s 117ms/step - loss: 7.7317e-04 - categorical_accuracy: 1.0000
Epoch 1931/2000
24/24 [==============================] - 2s 95ms/step - loss: 7.4759e-04 - categorical_accuracy: 1.0000
Epoch 1932/2000
24/24 [==============================] - 2s 95ms/step - loss: 7.1723e-04 - categorical_accuracy: 1.0000
Epoch 1933/2000
24/24 [==============================] - 2s 103ms/step - loss: 7.2504e-04 - categorical_accuracy: 1.0000
Epoch 1934/2000
24/24 [==============================] - 2s 100ms/step - loss: 6.7384e-04 - categorical_accuracy: 1.0000
Epoch 1935/2000
24/24 [==============================] - 2s 76ms/step - loss: 6.4944e-04 - categorical_accuracy: 1.0000
Epoch 1936/2000
24/24 [==============================] - 2s 84ms/step - loss: 6.2532e-04 - categorical_accuracy: 1.0000
Epoch 1937/2000
24/24 [=============================

In [166]:
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_15 (LSTM)               (None, 30, 64)            48896     
_________________________________________________________________
lstm_16 (LSTM)               (None, 30, 128)           98816     
_________________________________________________________________
lstm_17 (LSTM)               (None, 64)                49408     
_________________________________________________________________
dense_15 (Dense)             (None, 64)                4160      
_________________________________________________________________
dense_16 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_17 (Dense)             (None, 26)                858       
Total params: 204,218
Trainable params: 204,218
Non-trainable params: 0
________________________________________________

In [22]:
res = model.predict(X_test)

In [28]:
actions[np.argmax(res[4])]

'6'

In [29]:
actions[np.argmax(y_test[4])]

'Thumbs down'

In [25]:
import pickle

with open('model_pkl', 'wb') as files:
    pickle.dump(model, files)

TypeError: cannot pickle 'weakref' object

In [26]:
model.save('action.h5')

In [27]:
model.load_weights('action.h5')

In [30]:

from sklearn.metrics import multilabel_confusion_matrix, accuracy_score

In [31]:
yhat = model.predict(X_test)

In [32]:
ytrue = np.argmax(y_test, axis=1).tolist()
yhat = np.argmax(yhat, axis=1).tolist()

In [33]:


multilabel_confusion_matrix(ytrue, yhat)

array([[[36,  0],
        [ 3,  0]],

       [[36,  0],
        [ 3,  0]],

       [[35,  3],
        [ 1,  0]],

       [[38,  0],
        [ 1,  0]],

       [[11, 24],
        [ 1,  3]],

       [[37,  0],
        [ 2,  0]],

       [[37,  0],
        [ 2,  0]],

       [[38,  1],
        [ 0,  0]],

       [[36,  0],
        [ 3,  0]],

       [[37,  0],
        [ 2,  0]],

       [[38,  0],
        [ 1,  0]],

       [[38,  0],
        [ 1,  0]],

       [[36,  0],
        [ 3,  0]],

       [[35,  0],
        [ 4,  0]],

       [[37,  0],
        [ 2,  0]],

       [[33,  5],
        [ 1,  0]],

       [[36,  0],
        [ 3,  0]],

       [[37,  0],
        [ 2,  0]],

       [[36,  3],
        [ 0,  0]],

       [[38,  0],
        [ 1,  0]]], dtype=int64)

In [34]:
accuracy_score(ytrue, yhat)

0.07692307692307693

In [35]:
from scipy import stats

In [36]:
sequence = []
sentence = []
predictions = []
threshold = 0.5

cap = cv2.VideoCapture(0)
# Set mediapipe model 
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        # Read feed
        ret, frame = cap.read()

        # Make detections
        image, results = mediapipe_detection(frame, holistic)
        print(results)
        
        # Draw landmarks
        draw_styled_landmarks(image, results)
        
        # 2. Prediction logic
        keypoints = extract_keypoints(results)
        sequence.append(keypoints)
        sequence = sequence[-30:]
        
        if len(sequence) == 30:
            res = model.predict(np.expand_dims(sequence, axis=0))[0]
            sign = actions[np.argmax(res)]
            print(sign)
            cv2.putText(image, sign, (50,50), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 0, 255), 2, cv2.LINE_AA)
            
        # Show to screen
        cv2.imshow('OpenCV Feed', image)

        # Break gracefully
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.solution_base.SolutionOutputs'>
<class 'mediapipe.python.soluti

<class 'mediapipe.python.solution_base.SolutionOutputs'>
I Love You
<class 'mediapipe.python.solution_base.SolutionOutputs'>
I Love You
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base

0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs

0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs

0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs'>
0
<class 'mediapipe.python.solution_base.SolutionOutputs